In [1]:
# Import de bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pandas import Series
from datetime import datetime, timedelta
from pandas.tseries.offsets import DateOffset
from datetime import date
from datetime import timedelta
import holidays

from sklearn.impute import SimpleImputer
from cloudant import Cloudant

import warnings
warnings.filterwarnings("ignore")


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [2]:
# Definição das variáveis de conexão:
rodrigo = {'user': "",
           'password': "",
           'account': "",
           'db_name': ""
           }

In [3]:
# Funcao para carregar os dados do banco de dados NoSQL Cloudant
def obter_dados(user, password, account, db_name):

    client = Cloudant(user, password, account=account, connect=True, auto_renew=True)

    db = client[db_name]

    response = db.all_docs(limit=200000, include_docs=True)

    docs = []
    for r in response["rows"]:
        docs.append(r['doc'])

    return docs

# Verificando se existem registros duplicados no dataset
def verify_dup_rows(data):
    
    print("Linhas do dataset até o momento:")
    print(data.shape)

    # Obtendo somente linhas duplicadas
    tmp = data[data.duplicated()]

    print("\nLinhas duplicadas até o momento:")
    print(tmp.shape)
    
    del tmp
    
    
# Funcao para verificar valores missing no dataset
def check_missing(df, display = 10):
    temp_df = df.copy()
    df_nan = (temp_df.isnull().sum() / len(temp_df)) * 100
    missing_data = pd.DataFrame({'Missing n': temp_df.isnull().sum(),'% Missing' :df_nan})
    if missing_data['Missing n'].sum() == 0:
        return print('\nÓtimo! Não há mais valores faltantes neste dataset.')
    else:
        return missing_data.sort_values('% Missing', ascending = False).head(display)

# Funcao para preencher os dados missing de algumas colunas
def preenche_missing(df):
    
    df.dropna(axis='index', how='any', subset=['Tempo', 'Estação', 'LAT', 'LONG', 
                                               'Movimentação', 'Original_473', 
                                               'Original_269', 'Zero', 'Maçã-Verde', 
                                               'Tangerina', 'Citrus', 'Açaí-Guaraná', 
                                               'Pêssego'])

    # coloca constante 0 em outras colunas com dados missing
    impute_zeros = SimpleImputer(
                                    missing_values=np.nan,
                                    strategy='constant',
                                    fill_value=0,
                                    verbose=0,
                                    copy=True
                                )
    
    impute_zeros.fit(X=df)

    # Reconstruindo um Pandas DataFrame com os resultados
    data = pd.DataFrame.from_records(
            data = impute_zeros.transform(
            X    = df
        ),
        columns=df.columns
    )    
    
    # Converte colunas para numerico
    data = data.apply(pd.to_numeric, errors='ignore')

    return data

# Funcao para trabalhar as features
def feature_engineering(df, treino=True):

    if treino:
        
        # Transformando a coluna data
        df['Tempo'] = pd.to_datetime(df['Tempo'], errors='coerce')

        # Removendo colunas unicas
        df = df.drop(['_id', '_rev'], axis = 1)

        # Verificar os registros duplicados
        verify_dup_rows(df)

        # Removendo registros duplicados
        df = df.drop_duplicates().reset_index(drop=True)
    
        print('\nRename columns')
        # Renomeando colunas apenas para funcionar alguns algoritmos
        df = df.rename(columns={"Estação":"Estacao", "Movimentação":"Movimentacao", "Maçã-Verde": "Maca_Verde", "Açaí-Guaraná":"Acai_Guarana", "Pêssego":"Pessego"})        
        
    else:
        # Corrige dados incorretos
        teste.loc[teste.index == 89, 'Tempo'] = "2019-02-28 01:00:00"
        teste.loc[teste.index == 90, 'Tempo'] = "2019-02-28 02:00:00"

        teste.loc[teste.index == 449, 'Tempo'] = "2018-02-28 01:00:00"
        teste.loc[teste.index == 450, 'Tempo'] = "2018-02-28 02:00:00"

        teste.loc[teste.index == 809, 'Tempo'] = "2019-02-28 01:00:00"
        teste.loc[teste.index == 810, 'Tempo'] = "2019-02-28 02:00:00"
        
        # Transformando a coluna data
        df['Tempo'] = pd.to_datetime(df['Tempo'], errors='coerce')
        
        # Removendo colunas unicas
        df = df.drop(['LAT','LONG'], axis = 1)

    return df

# Funcao para transformar features categoricas em numericas
def fe_cat_num(df):

    # Tratando variáveis categóricas com o método Pandas ``get_dummies()''
    df = pd.get_dummies(df, columns=['Estação'])

    return df    

In [4]:
# Criando um dataframe a partir da lista de dados vindo do IoT e gravados no NoSQL do Walter:
train_walter = pd.DataFrame(data=obter_dados(walter['user'], walter['password'], walter['account'], walter['db_name']))
print(train_walter.shape)

(61466, 17)


In [5]:
# Criando um dataframe a partir da lista de dados vindo do IoT e gravados no NoSQL do Rodrigo:
train_rodrigo = pd.DataFrame(data=obter_dados(rodrigo['user'], rodrigo['password'], rodrigo['account'], rodrigo['db_name']))
print(train_rodrigo.shape)

(94390, 17)


In [6]:
# Carregando o dataset de testes 
train_evandro  = pd.read_csv(r'dataset_iot_evandro.csv')
print(train_evandro.shape)

(12413, 17)


In [7]:
# Concatenando as bases de dados:
train = pd.concat([train_walter, train_rodrigo, train_evandro], ignore_index=True)

In [8]:
print(train['row'].min())
print(train['row'].max())

1
17016


In [9]:
# Verificando as colunas com dados missing do dataset
check_missing(train, display = 15)


Ótimo! Não há mais valores faltantes neste dataset.


In [10]:
train.to_csv('desafio_7_treino.csv', index=False, encoding='utf-8')
train.head(10)

,_id,_rev,Tempo,Estação,LAT,LONG,Movimentação,Original_473,Original_269,Zero,Maçã-Verde,Tangerina,Citrus,Açaí-Guaraná,Pêssego,TARGET,row
0,001e999ea79df30784ef6542560ac2df,1-643c46dcabbe5a1e21fd1733108d083e,2019-12-30,Fradique Coutinho,-23.5661,-46.6841,35386,24,45,61,42,19,22,15,9,NORMAL,10800
1,001e999ea79df30784ef6542560b6139,1-68fc8f62b4ac8a66c898e9169b9b5d6e,2018-1-8,Luz,-23.5365,-46.6332,124806,15,10,10,29,10,25,14,13,NORMAL,10808
2,001e999ea79df30784ef654256119ef3,1-ed29d2eb3f09c1f4c4ba094c910f5b5a,2018-4-17,Luz,-23.5365,-46.6332,125215,53,1,55,18,18,6,37,8,REABASTECER,10907
3,001e999ea79df30784ef654256183917,1-67e32d4eb6c44b378820e05a3e71d02a,2018-7-18,Luz,-23.5365,-46.6332,126476,36,5,57,38,15,43,43,18,REABASTECER,10998
4,001e999ea79df30784ef6542561b2247,1-d1ffebf69ddc875ebb5276c32d87646c,2018-8-28,Luz,-23.5365,-46.6332,125917,79,10,8,37,32,30,18,15,NORMAL,11038
5,001e999ea79df30784ef65425620da23,1-d0c61b0026551e58d9535869044724e4,2018-10-5,Sé,-23.55,-46.6333,193894,12,23,38,8,6,8,14,6,REABASTECER,3875
6,001e999ea79df30784ef65425621f1ec,1-c091fff524c44d89e58d49ba693f8bba,2019-9-1,Luz,-23.5365,-46.6332,127370,86,63,65,7,6,33,32,9,NORMAL,11401
7,001e999ea79df30784ef65425625fd7f,1-7b042a2a7739ff8377e3f76692397c91,2019-2-1,Luz,-23.5365,-46.6332,127523,32,39,55,43,25,18,18,15,NORMAL,11191
8,001e999ea79df30784ef6542562a39b7,1-53016139f367080a96ea15fc82ebc712,2019-4-7,Luz,-23.5365,-46.6332,126574,86,24,58,30,18,31,19,7,NORMAL,11257
9,001e999ea79df30784ef6542563488c8,1-28aaac694572094e0bef6cadab0d24c4,2019-8-28,Luz,-23.5365,-46.6332,125128,22,13,25,13,19,38,40,18,NORMAL,11398


In [11]:
# Criando um dataframe a partir da lista de dados vindo do IoT e gravados no NoSQL
train = pd.read_csv(r'desafio_7_treino.csv')

# Feature Engineering
treino = preenche_missing(train)    
treino = feature_engineering(treino, True) 

# Verificando as colunas com dados missing do dataset
check_missing(treino, display = 15)

print('\nLinhas unicas até o momento:')
print(treino.shape)

treino.head(10)

Linhas do dataset até o momento:
(168269, 15)

Linhas duplicadas até o momento:
(151254, 15)

Rename columns

Linhas unicas até o momento:
(17015, 15)


,Tempo,Estacao,LAT,LONG,Movimentacao,Original_473,Original_269,Zero,Maca_Verde,Tangerina,Citrus,Acai_Guarana,Pessego,TARGET,row
0,2019-12-30,Fradique Coutinho,-23.5661,-46.6841,35386,24,45,61,42,19,22,15,9,NORMAL,10800
1,2018-01-08,Luz,-23.5365,-46.6332,124806,15,10,10,29,10,25,14,13,NORMAL,10808
2,2018-04-17,Luz,-23.5365,-46.6332,125215,53,1,55,18,18,6,37,8,REABASTECER,10907
3,2018-07-18,Luz,-23.5365,-46.6332,126476,36,5,57,38,15,43,43,18,REABASTECER,10998
4,2018-08-28,Luz,-23.5365,-46.6332,125917,79,10,8,37,32,30,18,15,NORMAL,11038
5,2018-10-05,Sé,-23.5500,-46.6333,193894,12,23,38,8,6,8,14,6,REABASTECER,3875
6,2019-09-01,Luz,-23.5365,-46.6332,127370,86,63,65,7,6,33,32,9,NORMAL,11401
7,2019-02-01,Luz,-23.5365,-46.6332,127523,32,39,55,43,25,18,18,15,NORMAL,11191
8,2019-04-07,Luz,-23.5365,-46.6332,126574,86,24,58,30,18,31,19,7,NORMAL,11257
9,2019-08-28,Luz,-23.5365,-46.6332,125128,22,13,25,13,19,38,40,18,NORMAL,11398


In [14]:
treino.to_csv('train_17015.csv', index=False, encoding='utf-8')
#treino.head(10)

In [13]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17015 entries, 0 to 17014
Data columns (total 15 columns):
Tempo           16921 non-null datetime64[ns]
Estacao         17015 non-null object
LAT             17015 non-null float64
LONG            17015 non-null float64
Movimentacao    17015 non-null int64
Original_473    17015 non-null int64
Original_269    17015 non-null int64
Zero            17015 non-null int64
Maca_Verde      17015 non-null int64
Tangerina       17015 non-null int64
Citrus          17015 non-null int64
Acai_Guarana    17015 non-null int64
Pessego         17015 non-null int64
TARGET          17015 non-null object
row             17015 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(10), object(2)
memory usage: 1.9+ MB
